In [2]:
import numpy as np
import pandas as pd
import torch
import zarr
import os
import matplotlib.pyplot as plt

from dredFISH.Utils import basicu

In [2]:
# f = "/bigstore/GeneralStorage/fangming/projects/dredfish/data/paintshop/mm10_isoflat_counts.csv"
f = "/bigstore/GeneralStorage/fangming/projects/dredfish/data/paintshop/mm10_isoflat_counts_filtered_Oct26.csv"
pshop = pd.read_csv(f, header=None)
genes_pshp = pshop[0].values
pshop

,0,1
0,0610005C13Rik,22
1,0610009B22Rik,8
2,0610009E02Rik,9
3,0610009L18Rik,10
4,0610010F05Rik,21
...,...,...
23373,Zyx,33
23374,Zzef1,214
23375,Zzz3,55
23376,a,10


In [3]:
f = "/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v4.zarr"
dataset = zarr.open(f, mode='r')
genes_data = dataset['genes'][:]
list(dataset.keys())

['counts',
 'genes',
 'l1_cat',
 'l1_code',
 'l2_cat',
 'l2_code',
 'l3_cat',
 'l3_code',
 'l5_cat',
 'l5_code']

In [4]:
genes_data

array(['0610005C13Rik', '0610006L08Rik', '0610007P14Rik', ..., 'n-R5s144',
       'n-R5s146', 'n-R5s149'], dtype='<U28')

In [5]:
# match and save idx and constrnts
genes_comm = np.intersect1d(genes_data, genes_pshp)
gidx_comm = basicu.get_index_from_array(genes_data, genes_comm)
genes_cnstrnts = pshop.set_index(0)[1].loc[genes_comm].values

print(genes_data.shape, genes_pshp.shape, genes_comm.shape, gidx_comm.shape, genes_cnstrnts.shape)
print(np.all(genes_comm == genes_data[gidx_comm]))
print(np.all(gidx_comm[1:] - gidx_comm[:-1] > 0)) # keep increasing
print(gidx_comm)
print(genes_comm)
print(genes_cnstrnts)
# plt.plot(gidx_comm)

(45768,) (23378,) (22038,) (22038,) (22038,)
True
True
[    0     3     4 ... 45739 45740 45741]
['0610005C13Rik' '0610009B22Rik' '0610009E02Rik' ... 'Zzef1' 'Zzz3' 'a']
[ 22   8   9 ... 214  55  10]


In [6]:
gidx_cnstrnts = torch.from_numpy(gidx_comm)
cnstrnts      = torch.from_numpy(genes_cnstrnts)

In [7]:
output_gidx_cnstrnts = os.path.join('/bigstore/GeneralStorage/fangming/projects/dredfish/data/', 'rna', 
                                    'gidx_cnstrnts_pshop_mm10_isoflat_filtered_Oct26.pt')
output_cnstrnts      = os.path.join('/bigstore/GeneralStorage/fangming/projects/dredfish/data/', 'rna',      
                                         'cnstrnts_pshop_mm10_isoflat_filtered_Oct26.pt')

In [8]:
torch.save(gidx_cnstrnts, output_gidx_cnstrnts)
torch.save(cnstrnts,      output_cnstrnts)

In [9]:
# check
a = torch.load(output_gidx_cnstrnts)
a

tensor([    0,     3,     4,  ..., 45739, 45740, 45741])

In [10]:
a.get_device()

-1

# compare the genes between the two versions

In [3]:
fa = os.path.join('/bigstore/GeneralStorage/fangming/projects/dredfish/data/', 'rna', 
                                    'gidx_cnstrnts_pshop_mm10_isoflat.pt')
fb = os.path.join('/bigstore/GeneralStorage/fangming/projects/dredfish/data/', 'rna',      
                                    'gidx_cnstrnts_pshop_mm10_isoflat_filtered_Oct26.pt')
idx_a = torch.load(fa)
idx_b = torch.load(fb)

In [4]:
idx_comm = np.intersect1d(idx_a, idx_b)
idx_a.shape, idx_b.shape, idx_comm.shape

(torch.Size([23282]), torch.Size([22038]), (22038,))